In [182]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pyproj
import geopandas as gpd
from shapely.geometry import Point
from sklearn.linear_model import Lasso,LassoCV,Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [190]:
for_temple = pd.read_csv("public_quantity_distance.csv")
for_temple['最近金融機構距離']
for_temple['最近廟宇距離']

# concat_data = pd.read_csv("new_concat_data.csv").drop(['最近金融機構距離','最近廟宇距離'], axis=1).reset_index(drop=True)
concat_data = pd.read_csv("new_concat_data.csv").reset_index(drop=True)
origin_concat_data = pd.read_csv("concat_data.csv")
origin_concat_data = origin_concat_data[11751:].drop(['geometry','縱坐標','橫坐標'], axis=1).reset_index(drop=True)
origin_concat_data['最近金融機構距離'] = for_temple['最近金融機構距離']
origin_concat_data['最近廟宇距離'] = for_temple['最近廟宇距離']

new_concat_data = pd.concat([concat_data, origin_concat_data], ignore_index=True)

In [191]:
Taipei = new_concat_data[new_concat_data['縣市'] == "台北市"]
Kaohsiung = new_concat_data[new_concat_data['縣市'] == "高雄市"]
New_Taipei = new_concat_data[new_concat_data['縣市'] == "新北市"]
Taoyuan = new_concat_data[new_concat_data['縣市'] == "桃園市"]
Taichung = new_concat_data[new_concat_data['縣市'] == "台中市"]
Tainan = new_concat_data[new_concat_data['縣市'] == "台南市"]
Hsinchu = new_concat_data[(new_concat_data['縣市'] == "新竹縣") | (new_concat_data['縣市'] == "新竹市")]
Else = new_concat_data[(new_concat_data['縣市'] != "台北市") & (new_concat_data['縣市'] != "高雄市") & (new_concat_data['縣市'] != "新北市") & (new_concat_data['縣市'] != "桃園市") & (new_concat_data['縣市'] != "台中市") &( new_concat_data['縣市'] != "台南市") & (new_concat_data['縣市'] != "新竹縣") & (new_concat_data['縣市'] != "新竹市")]
multi = [Taipei, Kaohsiung, New_Taipei, Taoyuan, Taichung, Tainan, Hsinchu, Else]

In [192]:
Taipei = Taipei[[
    '縣市', '鄉鎮市區', '路名', '主要用途', '主要建材', '建物型態', '使用分區', '備註', '單價',
    'lng', '最近航空站', '最近幼稚園、托兒所',
    '屋齡', '最近一般公園', '最近碼頭', '最近大型購物中心', 
    '最近超級市場', '最近大學及研究所', '總樓層數',
    '最近國中', '最近生活百貨量販',
    '最近捷運站', '最近電力資源', '最近高鐵站',
    '最近產業用水及設施', '最近飯店', '最近觀光遊憩設施', 
    '最近主要道路建設', 'lat', '最近里鄰公園', '最近娛樂設施', 
    '國中(高)', '國中(中)', '最近火車站', '車位面積', 
    '最近百貨公司', '陽台面積', '移轉層次', '最近停車場地', 
    '最近醫療保健', '最近金融機構', '最近加油站', 
    '土地面積', '主建物面積', '建物面積', '最近工廠', 
    '最近傳統市場', '附屬建物面積', '國中(下)', 
    '最近高中', '最近商店', '最近專科', '最近國小', 
    '最近廟宇距離', '最近金融機構距離'
]]
    
# Taipei.iloc[:, 9:] = pd.DataFrame(z_scaler.fit_transform(Taipei.iloc[:, 9:]))

In [207]:
z_scaler = preprocessing.StandardScaler()

def ClusterTrainer(city):
    
    # specific = concat_data[concat_data['縣市'] == city]
    specific = city
    
    specific = specific[[
        '縣市', '鄉鎮市區', '路名', '主要用途', '主要建材', '建物型態', '使用分區', '備註', '單價',
        'lng', '最近航空站', '最近幼稚園、托兒所',
        '屋齡', '最近一般公園', '最近碼頭', '最近大型購物中心', 
        '最近超級市場', '最近大學及研究所', '總樓層數',
        '最近國中', '最近生活百貨量販',
        '最近捷運站', '最近電力資源', '最近高鐵站',
        '最近產業用水及設施', '最近飯店', '最近觀光遊憩設施', 
        '最近主要道路建設', 'lat', '最近里鄰公園', '最近娛樂設施', 
        '國中(高)', '國中(中)', '最近火車站', '車位面積', 
        '最近百貨公司', '陽台面積', '移轉層次', '最近停車場地', 
        '最近醫療保健', '最近金融機構', '最近加油站', 
        '土地面積', '主建物面積', '建物面積', '最近工廠', 
        '最近傳統市場', '附屬建物面積', '國中(下)', 
        '最近高中', '最近商店', '最近專科', '最近國小', 
        '最近廟宇距離', '最近金融機構距離'
    ]]
    
    specific.iloc[:, 9:] = pd.DataFrame(z_scaler.fit_transform(specific.iloc[:, 9:]), index = specific.index)
    print(specific.shape)
    specific = pd.get_dummies(specific, columns=['鄉鎮市區','路名','主要用途','主要建材','建物型態','使用分區','備註'])
    specific = specific.drop(['縣市'], axis=1)

    # train_test_data = GetSplit(specific)
    Y_train = specific[['單價']].dropna(subset=['單價'])
    print(Y_train.shape)
    X_train = specific.dropna(subset=['單價']).drop(['單價'], axis=1)
    print(X_train.shape)
    X_test = specific[specific['單價'].isna()].drop(['單價'], axis=1)
    print(city['縣市'].unique())

    # ridge = Ridge()
    # parameters = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
    # ridge_reg = GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error',cv=5)
    # result = ridge_reg.fit(X_train.values, Y_train['單價'].values)
    rf_regressor = RandomForestRegressor()
    result = rf_regressor.fit(X_train.values, Y_train.values)
    
    return result, X_test

In [208]:
Taipei_res = ClusterTrainer(Taipei)
Kaohsiung_res = ClusterTrainer(Kaohsiung)
New_Taipei_res = ClusterTrainer(New_Taipei)
Taoyuan_res = ClusterTrainer(Taoyuan)
Taichung_res = ClusterTrainer(Taichung)
Tainan_res = ClusterTrainer(Tainan)
Hsinchu_res = ClusterTrainer(Hsinchu)
Else_res = ClusterTrainer(Else)

(2484, 55)
(1265, 1)
(1265, 601)
['台北市']


/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_12118/3224406145.py:44: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = rf_regressor.fit(X_train.values, Y_train.values)
/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_12118/3224406145.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  specific.iloc[:, 9:] = pd.DataFrame(z_scaler.fit_transform(specific.iloc[:, 9:]), index = specific.index)
/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_12118/3224406145.py:44: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example 

(1979, 55)
(1310, 1)
(1310, 706)
['高雄市']


/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_12118/3224406145.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  specific.iloc[:, 9:] = pd.DataFrame(z_scaler.fit_transform(specific.iloc[:, 9:]), index = specific.index)
/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_12118/3224406145.py:44: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = rf_regressor.fit(X_train.values, Y_train.values)


(6353, 55)
(4156, 1)
(4156, 1022)
['新北市']


...

(688, 55)
(474, 1)
(474, 299)
['台南市']


/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_12118/3224406145.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  specific.iloc[:, 9:] = pd.DataFrame(z_scaler.fit_transform(specific.iloc[:, 9:]), index = specific.index)
/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_12118/3224406145.py:44: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = rf_regressor.fit(X_train.values, Y_train.values)


(900, 55)
(590, 1)
(590, 375)
['新竹縣' '新竹市']


/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_12118/3224406145.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  specific.iloc[:, 9:] = pd.DataFrame(z_scaler.fit_transform(specific.iloc[:, 9:]), index = specific.index)
/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_12118/3224406145.py:44: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = rf_regressor.fit(X_train.values, Y_train.values)


(710, 55)
(477, 1)
(477, 414)
['苗栗縣' '基隆市' '屏東縣' '宜蘭縣' '花蓮縣' '嘉義市' '金門縣' '嘉義縣' '彰化縣' '雲林縣']


In [209]:
dic = {}

In [211]:
for i in range(len(Taipei_res[1])):
    lst = Taipei_res[0].predict(Taipei_res[1].values)
    dic[Taipei_res[1].index[i]] = lst[i]
    
for i in range(len(Kaohsiung_res[1])):
    lst = Kaohsiung_res[0].predict(Kaohsiung_res[1].values)
    dic[Kaohsiung_res[1].index[i]] = lst[i]
    
for i in range(len(New_Taipei_res[1])):
    lst = New_Taipei_res[0].predict(New_Taipei_res[1].values)
    dic[New_Taipei_res[1].index[i]] = lst[i]
    
for i in range(len(Taoyuan_res[1])):
    lst = Taoyuan_res[0].predict(Taoyuan_res[1].values)
    dic[Taoyuan_res[1].index[i]] = lst[i]
    
for i in range(len(Taichung_res[1])):
    lst = Taichung_res[0].predict(Taichung_res[1].values)
    dic[Taichung_res[1].index[i]] = lst[i]
    
for i in range(len(Tainan_res[1])):
    lst = Tainan_res[0].predict(Tainan_res[1].values)
    dic[Tainan_res[1].index[i]] = lst[i]
    
for i in range(len(Hsinchu_res[1])):
    lst = Hsinchu_res[0].predict(Hsinchu_res[1].values)
    dic[Hsinchu_res[1].index[i]] = lst[i]
    
for i in range(len(Else_res[1])):
    lst = Else_res[0].predict(Else_res[1].values)
    dic[Else_res[1].index[i]] = lst[i]

In [212]:
dic = sorted(dic.items(), key=lambda x: x[0])

In [215]:
lst = []
for item in dic:
    lst.append(item[1])

In [216]:
result = pd.read_csv("public_submission_template.csv")
result['predicted_price'] = lst
result.to_csv("./result.csv", index=False)